# Fast Training on full dataset
6th Apr

Model trains fine on tiny dataset - inspect initial training on full dataset with large batch size to see if it just blows up very early on. Turned off validation split to increase the logging speed.

In [1]:
import os
from dock2hit.utils import write_slurm_script

current_dir = os.getcwd()
data_dir = '/rds-d2/user/wjm41/hpc-work/datasets/Ugis/datasets'
log_dir = '/rds-d2/user/wjm41/hpc-work/datasets/Ugis/runs/ultra-large/D4-debug'

dataset='D4'
model_name = 'quick_train_ultra'
model_dir = f'/rds-d2/user/wjm41/hpc-work/models/dock2hit/{dataset}'

model_path = f'{model_dir}/{model_name}'
# data_path = f'{data_dir}/{dataset}_test.csv'
data_path = f'{data_dir}/{dataset}_docked_valid.pkl'

script = f'python -c "from dock2hit.train_and_validate_multithread import main; main()"'

log_step = 10
args = [
    f'-path_to_train_data {data_path}',
    '-batch_size 32768',
    '-minibatch_size 4096',
    '-n_epochs 10000',
    f'-save_dir {model_path}',
    f'-log_dir {log_dir}',
    f'-steps_per_log {log_step}'
    ]

file_name = 'subm_quick_train_ultra'
run_time = '0:15:00'
output_name = f'{current_dir}/subm_quick_train_ultra.out'

write_slurm_script(job_name=f'{dataset}_quick_train_ultra',
                   run_time=f'{run_time}',
                   output_name=output_name,
                   package_dir='/rds-d2/user/wjm41/hpc-work/datasets/Ugis/',
                   script=script,
                   args=args,
                   file_name=file_name,
                   email=True
                    )

print(f"Submitted quick_trainging jobs for {model_path} on {data_path}")

!sbatch {file_name}

/rds-d2/user/wjm41/hpc-work/conda/envs/dgl_cpu/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:rdkit:Enabling RDKit 2021.09.4 jupyter extensions


Submitted quick_trainging jobs for /rds-d2/user/wjm41/hpc-work/models/dock2hit/D4/quick_train_ultra on /rds-d2/user/wjm41/hpc-work/datasets/Ugis/datasets/D4_docked_valid.pkl
Submitted batch job 58478554


In [ ]:
!squeue -u wjm41


In [ ]:
!scancel 58464108


Seems to be working fine on tiny dataset:

![](tiny_training.png)

Perhaps exploding/vanishing gradients? Switch to LeakyRELU?